In [75]:
import nevergrad as ng
import numpy as np

In [12]:
def square(x, y=12):
    return sum((x - .5)**2) + abs(y)

In [14]:
optimizer = ng.optimizers.OnePlusOne(instrumentation=4, budget=100)

recommendation = optimizer.optimize(square)

print(recommendation)  # optimal args and kwargs

Candidate(args=(array([0.49436086, 0.49365343, 0.50145127, 0.46881927]),), kwargs={})


### Full list of optimizers

In [15]:
# print(list(sorted(ng.optimizers.registry.keys())))

In [47]:
instrum = ng.Instrumentation(ng.var.Array(2), y=ng.var.Array(1).asscalar())

optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=100)

recommendation = optimizer.minimize(square)

print(recommendation)


Candidate(args=(array([0.47523085, 0.53453307]),), kwargs={'y': -0.00015505662801493502})


In [49]:
# argument transformation
arg1 = ng.var.OrderedDiscrete(["a", "b"])  # 1st arg. = positional discrete argument
arg2 = ng.var.SoftmaxCategorical(["a", "c", "e"])  # 2nd arg. = positional discrete argument
value = ng.var.Gaussian(mean=1, std=2)  # the 4th arg. is a keyword argument with Gaussian prior

# create the instrumented function
instrum = ng.Instrumentation(arg1, arg2, "blublu", value=value)
# the 3rd arg. is a positional arg. which will be kept constant to "blublu"

print(instrum.dimension)  # 5 dimensional space

# The dimension is 5 because:
# - the 1st discrete variable has 1 possible values, represented by a hard thresholding in
#   a 1-dimensional space, i.e. we add 1 coordinate to the continuous problem
# - the 2nd discrete variable has 3 possible values, represented by softmax, i.e. we add 3 coordinates to the continuous problem
# - the 3rd variable has no uncertainty, so it does not introduce any coordinate in the continuous problem
# - the 4th variable is a real number, represented by single coordinate.

print(instrum.data_to_arguments([1, -80, -80, 80, 3]))
# prints (args, kwargs): (('b', 'e', 'blublu'), {'value': 7})
# b is selected because 1 > 0 (the threshold is 0 here since there are 2 values.
# e is selected because proba(e) = exp(80) / (exp(80) + exp(-80) + exp(-80))
# value=7 because 3 * std + mean = 7

5
(('b', 'e', 'blublu'), {'value': 7})


In [55]:
def myfunction(arg1, arg2, arg3, value=3):
    print(arg1, arg2, arg3)
    return value**2

optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=5)
recommendation = optimizer.minimize(myfunction)

print(recommendation)

a e blublu
a a blublu
b e blublu
a e blublu
b c blublu
Candidate(args=('a', 'a', 'blublu'), kwargs={'value': 1.0})


In [56]:
# let us instantiate a fake recommendation for the sake of this tutorial
recommendation = optimizer.create_candidate.from_data([1, -80, -80, 80, -.5], deterministic=True)
print(recommendation.args)    # should print ["b", "e", "blublu"]
print(recommendation.kwargs)  # should print {"value": 0} because -.5 * std + mean = 0

# but be careful, since some variables are stochastic (SoftmaxCategorical ones are), setting 
# deterministic=False may yield different results
# The following will print more information on the conversion to your arguments:
print(instrum.get_summary(recommendation.data))

('b', 'e', 'blublu')
{'value': 0.0}
 - arg #1: Value b, from data: 1.0
 - arg #2: Value e, from data: [-80. -80.  80.] yielding probas: "a": 0.0%, "c": 0.0%, "e": 100.0%
 - kwarg "value": Value 0.0, from data: -0.5


In [65]:
instrum = ng.Instrumentation(ng.var.Array(2), y=ng.var.Array(1).asscalar())
optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=100)
recommendation = optimizer.minimize(square)
print(recommendation)

Candidate(args=(array([0.53707801, 0.49329983]),), kwargs={'y': 0.0009697290572580983})


In [133]:
from concurrent import futures
# instrum.random_state.seed(12)
optimizer = ng.optimizers.RandomSearch(instrumentation=instrum, budget=100, num_workers=6)
with futures.ProcessPoolExecutor(max_workers=optimizer.num_workers) as executor:
    recommendation = optimizer.minimize(square, executor=executor, batch_mode=False)
    print(recommendation)
    
square(recommendation.args[0], y=recommendation.kwargs['y'])

Candidate(args=(array([0.5159646 , 0.27462322]),), kwargs={'y': -0.29969637986698916})


0.35074594084373506

### Nevergrad for machine learning

In [106]:
def myfunction(lr, num_layers, arg3, arg4, other_anything):
    # something to minimize
    return -accuracy  

In [107]:
# instrument learning rate and number of layers, keep arg3 to 3 and arg4 to 4

lr = ng.var.Array(1).asscalar().bounded(0, 3).exponentiated(base=10, coeff=-1)
# log distributed between 0.001 and 1
num_layers = ng.var.OrderedDiscrete([4, 5, 6])

instrumentation = ng.Instrumentation(lr, num_layers, 3., arg4=4)

In [116]:
# check default value (initial guess)
args, kwargs = instrumentation.data_to_arguments([0] * instrumentation.dimension)
print(args, kwargs)

(0.03162277660168379, 5, 3.0) {'arg4': 4}


In [168]:
instrum = ng.Instrumentation(ng.var.Array(2), y=ng.var.Array(1).asscalar())
optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=100, num_workers=1)
optimizer.budget = 10

for _ in range(optimizer.budget):
    x = optimizer.ask()
    value = square(*x.args, **x.kwargs)
    print('{}: {}'.format(x, value))
    optimizer.tell(x, value)
recommendation = optimizer.provide_recommendation()


value = square(*recommendation.args, **recommendation.kwargs)
print('\n{}: {}'.format(recommendation, value))

Candidate(args=(array([0., 0.]),), kwargs={'y': 0.0}): 0.5
Candidate(args=(array([3.94657082, 1.08371297]),), kwargs={'y': 1.2601486642916229}): 13.479719901088748
Candidate(args=(array([0.48692353, 0.82116336]),), kwargs={'y': -1.6807666147404596}): 1.7840835114515208
Candidate(args=(array([-0.02870206, -0.49399489]),), kwargs={'y': 0.10927790078856424}): 1.3768296210914008
Candidate(args=(array([-0.9042828 ,  0.24156515]),), kwargs={'y': 1.1582471202597369}): 3.1970458648996107
Candidate(args=(array([0.30220881, 0.30809075]),), kwargs={'y': -0.42969006598419757}): 0.5056405826356624
Candidate(args=(array([-0.89244681, -0.71316235]),), kwargs={'y': -1.002554920618855}): 4.4132259457746805
Candidate(args=(array([-0.84976284, -0.35743159]),), kwargs={'y': -0.39650395076958955}): 2.9535526055951484
Candidate(args=(array([0.39006292, 1.09041546]),), kwargs={'y': 0.847119889290741}): 1.2077964700840498
Candidate(args=(array([0.19754312, 0.11974599]),), kwargs={'y': -0.14741628822292122}): 

### First example (Optimization of continuous hyperparameters)

In [214]:
def train_and_return_test_error(x):
    return np.linalg.norm([int(50. * abs(x_ - 0.2)) for x_ in x])

instrumentation = ng.Instrumentation(ng.var.Array(300))  # optimize on R^300

budget = 1200  # How many trainings we will do before concluding.

names = ['RandomSearch', 'OnePlusOne', 'TwoPointsDE', 'PSO']

In [215]:
for name in names:
    optim = ng.optimizers.registry[name](instrumentation=instrumentation, budget=budget)
    for u in range(budget // 3):      
        x1 = optim.ask()
        # Ask and tell can be asynchronous.
        # Just be careful that you "tell" something that was asked.
        # Here we ask 3 times and tell 3 times in order to fake asynchronicity
        x2 = optim.ask()
        x3 = optim.ask()
        # The three folowing lines could be parallelized.
        # We could also do things asynchronously, i.e. do one more ask
        # as soon as a training is over.       
        y1 = train_and_return_test_error(*x1.args, **x1.kwargs)  # here we only defined an arg, so   
        y2 = train_and_return_test_error(*x2.args, **x2.kwargs)  # we could omit kwargs (keeping it
        y3 = train_and_return_test_error(*x3.args, **x3.kwargs)  # here for the sake of consistency)
        optim.tell(x1, y1)
        optim.tell(x2, y2)
        optim.tell(x3, y3)
   
    recommendation = optim.recommend();
    if name is 'PSO': name = 'PSO\t'
    print('• {} \tprovides a vector of parameters with test error {:7.2f}'.format(
          name, train_and_return_test_error(*recommendation.args, **recommendation.kwargs)))

• RandomSearch 	provides a vector of parameters with test error  758.99
• OnePlusOne 	provides a vector of parameters with test error  162.17
• TwoPointsDE 	provides a vector of parameters with test error  383.16
• PSO	 	provides a vector of parameters with test error 5537.31


In [206]:
from concurrent import futures

for name in names:
    optim = ng.optimizers.registry[name](instrumentation=instrumentation, budget=budget,
                                        num_workers=10)
    # the executor will evaluate the function in multiple threads
    with futures.ThreadPoolExecutor(max_workers=optim.num_workers) as executor:  
        recommendation = optim.minimize(train_and_return_test_error, executor=executor)
    if name is 'PSO': name = 'PSO\t'
    print('• {} \tprovides a vector of parameters with test error {:7.2f}'.format(
          name, train_and_return_test_error(*recommendation.args, **recommendation.kwargs)))

• RandomSearch 	provides a vector of parameters with test error  761.41
• OnePlusOne 	provides a vector of parameters with test error  167.13
• TwoPointsDE 	provides a vector of parameters with test error  400.04
• PSO	 	provides a vector of parameters with test error 4532.58


### Second example: optimization of mixed (continuous and discrete) hyperparameters.

In [208]:
# Let us define a function.
def myfunction(arg1, arg2, arg3, value=3):
    return np.abs(value) + (1 if arg1 != "a" else 0) + (1 if arg2 != "e" else 0)

# This function must then be instrumented in order to let the optimizer now what are the arguments.

In [218]:
# argument transformation
# Optimization of mixed (continuous and discrete) hyperparameters.

arg1 = ng.var.OrderedDiscrete(["a", "b"])  # 1st arg. = positional discrete argument

# We apply a softmax for converting real numbers to discrete values.
arg2 = ng.var.SoftmaxCategorical(["a", "c", "e"])  # 2nd arg. = positional discrete argument

value = ng.var.Gaussian(mean=1, std=2)  # the 4th arg. is a keyword argument with Gaussian prior

# create the instrumentation
# the 3rd arg. is a positional arg. which will be kept constant to "blublu"
instrumentation = ng.Instrumentation(arg1, arg2, "blublu", value=value)

print(instrumentation.dimension)  # 5 dimensional space

5


In [219]:
args, kwargs = instrumentation.data_to_arguments([1, -80, -80, 80, 3])
print(args, kwargs)

output = myfunction(*args, **kwargs)

print(output)

('b', 'e', 'blublu') {'value': 7}
8


In [236]:
budget = 1200  # How many episode we will do before concluding.
names = ['RandomSearch', 'ScrHammersleySearch', 'TwoPointsDE', 
         'OnePlusOne', 'PortfolioDiscreteOnePlusOne', 'PSO']
for name in names:
    optim = ng.optimizers.registry[name](instrumentation=instrumentation, budget=budget)
    for u in range(budget // 3):
        x1 = optim.ask()
        x2 = optim.ask()
        x3 = optim.ask()

        y1 = myfunction(*x1.args, **x1.kwargs)  # here we only defined an arg, so   
        y2 = myfunction(*x2.args, **x2.kwargs)  # we could omit kwargs (keeping it
        y3 = myfunction(*x3.args, **x3.kwargs)  # here for the sake of consistency)
        optim.tell(x1, y1)
        optim.tell(x2, y2)
        optim.tell(x3, y3)
    recommendation = optim.recommend()
    name = name + (len(max(names, key=len)) // 8-len(name)//8)*'\t'
    print('• {} \tprovides a vector of parameters with test error {:5.2f}'.format(
          name, myfunction(*recommendation.args, **recommendation.kwargs)))


• RandomSearch		 	provides a vector of parameters with test error  1.00
• ScrHammersleySearch	 	provides a vector of parameters with test error  1.01
• TwoPointsDE		 	provides a vector of parameters with test error  0.00
• OnePlusOne		 	provides a vector of parameters with test error  1.00
• PortfolioDiscreteOnePlusOne 	provides a vector of parameters with test error  1.00
• PSO			 	provides a vector of parameters with test error  0.00


In [237]:
# You always have the possibility to define your own instrumentation 
# inside your function (not recommended):

def softmax(x, possible_values=None):
    expx = [np.exp(x_ - max(x)) for x_ in x]
    probas = [e / sum(expx) for e in expx]
    return np.random.choice(len(x) if possible_values is None
            else possible_values, size=1, p=probas)


def train_and_return_test_error_mixed(x):
    cx = [x_ - 0.1 for x_ in x[3:]]
    activation = softmax(x[:3], ["tanh", "sigmoid", "relu"])
    return np.linalg.norm(cx) + (1. if activation != "tanh" else 0.)

instrumentation = 10  # you can just provide the size of your input in this case

#This version is bigger.
def train_and_return_test_error_mixed(x):
    cx = x[:(len(x) // 2)]  # continuous part.
    presoftmax_values = x[(len(x) // 2):]  # discrete part.
    values_for_this_softmax = []
    dx = []
    for g in presoftmax:
        values_for_this_softmax += [g]
        if len(values_for_this_softmax) > 4:
            dx += softmax(values_for_this_softmax)
            values_for_this_softmax = []
    return np.linalg.norm([int(50. * abs(x_ - 0.2)) for x_ in cx]) + [
            1 if d != 1 else 0 for d in dx]

instrumentation = 300